In [ ]:
# убеждаемся, что используем видеокарту
!nvidia-smi

In [ ]:
# Подключаем библиотеки
import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
import torch.nn.functional as F

In [ ]:
# Переключаем cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Клонируем датасет
!git clone https://github.com/jaddoescad/ants_and_bees

In [ ]:
!ls ants_and_bees

In [ ]:
# Загружаем датасет
transform_train = transforms.Compose([transforms.Resize([224,224]),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomAffine(0,shear = 10,scale=(.8,1.2)),
                                transforms.ColorJitter(brightness =1,saturation = 1,contrast = 1),
                                transforms.ToTensor(),
                                transforms.Normalize([.5,],[.5,])])


transform = transforms.Compose([transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize([.5,],[.5,])])
                                

training_dataset = datasets.ImageFolder('ants_and_bees/train',transform =transform_train)
validation_dataset = datasets.ImageFolder('ants_and_bees/val',transform =transform)

training_loader = torch.utils.data.DataLoader(training_dataset,batch_size = 20,shuffle =True)
validation_loader = torch.utils.data.DataLoader(validation_dataset,batch_size = 20,shuffle =False)

In [ ]:
# Функция для конвертации
def im_convert(tensor):
  image = tensor.clone().detach().numpy()
  image = image.transpose(1,2,0)
  print(image.shape)
  image = image*np.array([.5,.5,.5])+np.array([.5,.5,.5])
  image = image.clip(0,1)
  return image

In [ ]:
# выводим картинки
classes = ('ant', 'bee')
dataiter  = iter(training_loader)
images,labels = dataiter.next()
fig = plt.figure(figsize=(25,4))

for idx in np.arange(20):
  ax = fig.add_subplot(2,10,idx+1,xticks=[],yticks=[])
  plt.imshow(im_convert(images[idx]))
  ax.set_title(classes[labels[idx].item()])

In [ ]:
# загружаем модель
model = models.vgg16(pretrained = True)
print(model)

In [ ]:
# Указываем параметры
for param in model.features.parameters():
  param.requires_grad = False

In [ ]:
# 
n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs,len(classes))
model.classifier[6] = last_layer
model.to(device)

In [ ]:
# Обучение сети

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=.0001)

epochs =10
running_loss_history = []
running_corrects_history = []
val_running_loss_history = []
val_running_corrects_history = []

for i in range(epochs):
  running_loss = 0.0
  running_corrects= 0.0
  val_running_loss = 0.0
  val_running_corrects= 0.0
  for inputs,labels in training_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    outputs = model(inputs)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _,preds = torch.max(outputs,1)
   
    running_loss += loss.item()
    running_corrects += torch.sum(preds==labels.data)

  else:
    with torch.no_grad():

      for val_inputs,val_labels in validation_loader:
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
        
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs,val_labels)

        _,val_preds = torch.max(val_outputs,1)

        val_running_loss += val_loss.item()
        val_running_corrects += torch.sum(val_preds==val_labels.data)

    epoch_loss = running_loss/len(training_loader.dataset)*100
    epoch_acc =running_corrects.float()/len(training_loader.dataset)*100
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc)
    print('epoch :',i+1)
    print('training loss : {:.4f},training_acc :{:.4f}'.format(epoch_loss,epoch_acc.item()))

    val_epoch_loss = val_running_loss/len(validation_loader.dataset)*100
    val_epoch_acc = val_running_corrects.float()/len(validation_loader.dataset)*100
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc)
    print('validation loss : {:.4f},val_acc : {:.4f}'.format(val_epoch_loss,val_epoch_acc.item()))

In [ ]:
plt.plot(running_loss_history)
plt.plot(val_running_loss_history)

In [ ]:
plt.plot(running_corrects_history)
plt.plot(val_running_corrects_history)

In [ ]:
dataiter2  = iter(validation_loader)
images,labels = dataiter2.next()
images_ = images.to(device)
labels = labels.to(device)


fig = plt.figure(figsize=(25,4))
output = model(images_)
_, preds = torch.max(output, 1)
for idx in np.arange(20):
  ax = fig.add_subplot(2,10,idx+1,xticks=[],yticks=[])
  plt.imshow(im_convert(images[idx]))
  ax.set_title("{} ({}) ".format(str(classes[preds[idx].item()]),str(classes[labels[idx].item()])),color = ("green" if preds[idx].item()==labels[idx] else "red"))